# KNN

In [65]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import os
import math
import tqdm

In [66]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

In [67]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
for col in asthma_df.loc[:, "HR00":"HR23"]:
    for i in range(len(asthma_df[col])):
        if (asthma_df.loc[i, col] > 1000):
            # Take first 2 digits of float
            first_2_digits = str(asthma_df.loc[i, col])[:2]
            temp_number = float(first_2_digits)
            # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
            if temp_number < 20:
                val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                asthma_df.loc[i, col] = float(val)
            else:
                val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                asthma_df.loc[i, col] = float(val)
            pass


In [68]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    random_index = np.random.randint(0, len(df))
    random_column = columns[np.random.randint(0, len(columns))]

    df.loc[random_index, random_column]

    while pd.isna(df.loc[random_index, random_column]):
        random_index = np.random.randint(0, len(df))
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column]

In [69]:
get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])

[2215, 'HR03']

In [76]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data
k = []
for i in range(1, 6):
    difference = []
    for i in tqdm.tqdm(range(10000)):
        random_index = get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                    "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
        tmp = asthma_df.loc[random_index[0], random_index[1]]
        asthma_df.loc[random_index[0], random_index[1]] = np.nan
        imputer = KNNImputer()          # n_neighbors=2
        asthma_df.loc[:, "HR00":"HR23"] = imputer.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
        # get the absolute value of the difference between the original value and the imputed value
        difference.append(abs(tmp - asthma_df.loc[random_index[0], random_index[1]]))
    k.append(math.sum(difference))


print(k)

  6%|▌         | 600/10000 [00:11<02:53, 54.14it/s]


KeyboardInterrupt: 